In [1]:
import os

dir_notebook = os.getcwd()
print(dir_notebook)

/home/david/proyectos4/repositorios/Proyectos-SGBA1/Pruebas/Modelos/precio_peninsula


In [2]:
# mostrar las tarjetas graficas disponibles para usar con tensorflow
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2025-03-26 20:00:00.796284: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-26 20:00:00.965696: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-26 20:00:00.966877: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-26 20:00:01.926488: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12818613695195345681
xla_global_id: -1
]


2025-03-26 20:00:04.188193: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-26 20:00:04.188607: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
ruta_trabajo = '../../../'

# Cambiar el directorio de trabajo
os.chdir(ruta_trabajo)

print("Directorio de trabajo actual:", os.getcwd())

Directorio de trabajo actual: /home/david/proyectos4/repositorios/Proyectos-SGBA1


In [4]:
import pandas as pd
df = pd.read_parquet('data/processed/datos_precio/clima_precio_merged_recortado.parquet')

In [5]:
df.head()

,timestamp,€/kwh,fecha,tmed_3196,tmin_3196,tmax_3196,velmedia_3196,racha_3196,sol_3196,presMax_3196,...,horaPresMax_cos_8175,horaPresMin_varias_8175,horaPresMin_minutos_8175,horaPresMin_sin_8175,horaPresMin_cos_8175,prec_inapreciable_8175,prec_valor_8175,prec_log_8175,dir_sin_8175,dir_cos_8175
0,2014-11-03 14:00:00,0.0,2014-11-03,11.5,8.0,15.0,6.1,17.5,1.1,938.3,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
1,2014-11-03 15:00:00,0.0,2014-11-03,11.5,8.0,15.0,6.1,17.5,1.1,938.3,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
2,2014-11-03 16:00:00,0.0,2014-11-03,11.5,8.0,15.0,6.1,17.5,1.1,938.3,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
3,2014-11-03 17:00:00,0.0,2014-11-03,11.5,8.0,15.0,6.1,17.5,1.1,938.3,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
4,2014-11-03 18:00:00,0.0,2014-11-03,11.5,8.0,15.0,6.1,17.5,1.1,938.3,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648


In [6]:
for column in df.columns:
    print(f"columna:{column}, tipo:{df[column].dtype}")

columna:timestamp, tipo:datetime64[ns]
columna:€/kwh, tipo:float64
columna:fecha, tipo:datetime64[ns]
columna:tmed_3196, tipo:float64
columna:tmin_3196, tipo:float64
columna:tmax_3196, tipo:float64
columna:velmedia_3196, tipo:float64
columna:racha_3196, tipo:float64
columna:sol_3196, tipo:float64
columna:presMax_3196, tipo:float64
columna:presMin_3196, tipo:float64
columna:anio, tipo:float64
columna:mes, tipo:float64
columna:dia, tipo:float64
columna:dia_semana, tipo:float64
columna:estacion, tipo:float64
columna:horatmin_varias_3196, tipo:float64
columna:horatmin_minutos_3196, tipo:float64
columna:horatmin_sin_3196, tipo:float64
columna:horatmin_cos_3196, tipo:float64
columna:horatmax_varias_3196, tipo:float64
columna:horatmax_minutos_3196, tipo:float64
columna:horatmax_sin_3196, tipo:float64
columna:horatmax_cos_3196, tipo:float64
columna:horaracha_varias_3196, tipo:float64
columna:horaracha_minutos_3196, tipo:float64
columna:horaracha_sin_3196, tipo:float64
columna:horaracha_cos_319

In [7]:
# Eliminar todas las columnas que no contengan "temperatura" "viento" y "sol" en el nombre de la columna 

df_reducido = df.copy()
lista_particulas_mantener = [
    "tm", 
    "viento", 
    "sol", 
    "€/kwh", 
    "fecha", 
    "anio",
    "mes",
    "dia",
    "dia_semana",
    "estacion"]
for column in df.columns:
    if not any([particula in column for particula in lista_particulas_mantener]):
        df_reducido.drop(column, axis=1, inplace=True)

In [8]:
df_reducido.head()

,timestamp,€/kwh,fecha,tmed_3196,tmin_3196,tmax_3196,velmedia_3196,sol_3196,anio,mes,...,velmedia_8175,sol_8175,horatmin_varias_8175,horatmin_minutos_8175,horatmin_sin_8175,horatmin_cos_8175,horatmax_varias_8175,horatmax_minutos_8175,horatmax_sin_8175,horatmax_cos_8175
0,2014-11-03 14:00:00,0.0,2014-11-03,11.5,8.0,15.0,6.1,1.1,2014.0,11.0,...,5.0,9.1,0.0,420.0,0.965926,-0.258819,0.0,820.0,-0.422618,-0.906308
1,2014-11-03 15:00:00,0.0,2014-11-03,11.5,8.0,15.0,6.1,1.1,2014.0,11.0,...,5.0,9.1,0.0,420.0,0.965926,-0.258819,0.0,820.0,-0.422618,-0.906308
2,2014-11-03 16:00:00,0.0,2014-11-03,11.5,8.0,15.0,6.1,1.1,2014.0,11.0,...,5.0,9.1,0.0,420.0,0.965926,-0.258819,0.0,820.0,-0.422618,-0.906308
3,2014-11-03 17:00:00,0.0,2014-11-03,11.5,8.0,15.0,6.1,1.1,2014.0,11.0,...,5.0,9.1,0.0,420.0,0.965926,-0.258819,0.0,820.0,-0.422618,-0.906308
4,2014-11-03 18:00:00,0.0,2014-11-03,11.5,8.0,15.0,6.1,1.1,2014.0,11.0,...,5.0,9.1,0.0,420.0,0.965926,-0.258819,0.0,820.0,-0.422618,-0.906308


In [9]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

# Prepare the data
# Remove datetime columns before scaling
features = df.select_dtypes(exclude=['datetime64[ns]'])
target = df['€/kwh']

# Scale the features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

In [11]:
# Create sequences for LSTM (using last 24 time steps to predict next value)
def create_sequences(data, seq_length=24):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length)])
        y.append(data[i + seq_length, 0])  # Assuming price is first column after scaling
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_features)


In [12]:



# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create LSTM model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# Make predictions
predictions = model.predict(X_test)

# Inverse transform predictions to original scale
# You'll need to reshape and add other columns back to match original scaling
predictions_reshaped = np.zeros((len(predictions), features.shape[1]))
predictions_reshaped[:, 0] = predictions.flatten()
predictions_original = scaler.inverse_transform(predictions_reshaped)[:, 0]

: 